In [22]:
real_window_size_1gram = 5
real_window_size_class = 3

window_size_class = (real_window_size_class * 2) + 1
window_size_1gram = (real_window_size_1gram * 2) + 1

using_1gram = 1
using_2gram = 1
using_3gram = 0
using_5gram = 0
using_freqwords_eachclass = 0

#######################################################################################################
from data_handler import *
from feature_design_for_ngrams import *

pre_X_train = load('pre_X_train') # preprocessed text data
pre_X_test = load('pre_X_test') # preprocessed text data

word_one_gram_lookup_table = load('word_one_gram_lookup_table')
word_bi_gram_lookup_table = load('word_bi_gram_lookup_table')
word_tri_gram_lookup_table = load('word_tri_gram_lookup_table')
word_five_gram_lookup_table = load('word_five_gram_lookup_table')

lookup_table_class_component = load('lookup_table_class_component')
lookup_table_class_refinement_of_component = load('lookup_table_class_refinement_of_component')
lookup_table_class_action = load('lookup_table_class_action')
lookup_table_class_refinement_of_action = load('lookup_table_class_refinement_of_action')
lookup_table_class_condition = load('lookup_table_class_condition')
lookup_table_class_priority = load('lookup_table_class_priority')
lookup_table_class_motivation = load('lookup_table_class_motivation')
lookup_table_class_role = load('lookup_table_class_role')
lookup_table_class_object = load('lookup_table_class_object')
lookup_table_class_refinement_of_object = load('lookup_table_class_refinement_of_object')
lookup_table_class_sub_action = load('lookup_table_class_sub_action')
lookup_table_class_sub_argument_of_action = load('lookup_table_class_sub_argument_of_action')
lookup_table_class_sub_priority = load('lookup_table_class_sub_priority')
lookup_table_class_sub_role = load('lookup_table_class_sub_role')
lookup_table_class_sub_object = load('lookup_table_class_sub_object')
lookup_table_class_sub_refinement_of_object = load('lookup_table_class_sub_refinement_of_object')
lookup_table_class_none = load('lookup_table_class_none')

### FE_X_train와 FE_X_test 초기화
# FE_X_data는 feature extraction을 통해서 text에서 numerical vector로 변환된다.
FE_X_train = []
for sentence in (pre_X_train):
    temp = []
    for token in (sentence):
        temp.append('point_to_be_vector_via_feature_extraction')
    FE_X_train.append(temp)

FE_X_test = []
for sentence in (pre_X_test):
    temp = []
    for token in (sentence):
        temp.append('point_to_be_vector_via_feature_extraction')
    FE_X_test.append(temp)

# for feature information...
feature_info_without_LK = ['###### ************** Feature Information ************** ######']
this_feature_length = 0

# Feature Extraction 
### based on lookup table or without *linguistic knowledge*

* feature extraction할 때, numbering함수를 통해서 unknown_token/phrase의 포함여부를 결정지을 수 있다.
* 상대적으로 크기가 작은 lookup table은 unknown을 포함시키지 말자. (-> 실험적으로 판단하자. 몇몇의 실험에서는 성능이 조금 더 증가하였다.)
* high-dimensional and sparse feature를 더 압축하고 abstraction할 수 있는 도구인 PCA를 사용할 수 있을지 생각해보자.

### Step 1. Using word one-gram lookup table 
* 기본적으로 current time의 token만 고려하지만,
* window 개념을 도입하면, 그 size에 만큼 feature를 더 확장할 수 있다.
* 우선은 current time의 token만 사용하자

* 만약 window size가 3이라면 다음과 같이 feature를 구성한다.
   - [ token(t-1), token(t), token(t+1) ]

In [23]:
if using_1gram == 1:
    # for training data
    for i, sentence in enumerate(pre_X_train):
        for j, token in enumerate(sentence):
            FE_X_train[i][j] = feature_extraction_1gram(j, sentence, word_one_gram_lookup_table, window_size_1gram) 
            this_feature_length = len(FE_X_train[i][j])

    # for testing data
    for i, sentence in enumerate(pre_X_test):
        for j, token in enumerate(sentence):
            FE_X_test[i][j] = feature_extraction_1gram(j, sentence, word_one_gram_lookup_table, window_size_1gram)                 

    # Write feature information
    #feature_info_without_LK.append('word one gram (t) and dim = '+str(this_feature_length))
    feature_info_without_LK.append('=> word one gram window size'+ str(window_size_1gram) +'and dim = '+str(this_feature_length))
    print('feature length 1-gram = ', this_feature_length)

feature length 1-gram =  8019


주의 할점: Step1같은 경우 처음으로 feature vector를 만드는 단계이므로 FE_X_test[i][j] = step1_feature_vector와 같이 feature vector에 할당한다. 하지만, 그 이후의 단계에서는 feature vector를 += 와 같이 concatenation해야 한다. 

### Step 2. Using word bi-gram lookup table 

* 크기가 2이기 때문에 언제나 비대칭을 이룬다. 
* 따라서, 다음과 같이 2개의 경우가 생긴다
   - [ token(t-1), token(t) ]
   - [ token(t), token(t+1) ]

In [24]:
if using_2gram == 1:
    # for training data
    for i, sentence in enumerate(pre_X_train):
        for j, token in enumerate(sentence):
            step2_feature_vector = [] # [ 1g_t(t-1), 1g_t(t), 1g_t(t+1) ]

            """ [ token(t-1), token(t) ] """
            step2_feature_vector += numbering_token_using_2gram_lookup(j-1, j, sentence, word_bi_gram_lookup_table)
            """ [ token(t), token(t-1) ] """
            step2_feature_vector += numbering_token_using_2gram_lookup(j, j+1, sentence, word_bi_gram_lookup_table)

            FE_X_train[i][j] += step2_feature_vector # feature assignment
            this_feature_length = len(step2_feature_vector)

    # for testing data
    for i, sentence in enumerate(pre_X_test):
        for j, token in enumerate(sentence):
            step2_feature_vector = [] # [ 1g_t(t-1), 1g_t(t), 1g_t(t+1) ]

            """ [ token(t-1), token(t) ] """
            step2_feature_vector += numbering_token_using_2gram_lookup(j-1, j, sentence, word_bi_gram_lookup_table)
            """ [ token(t), token(t-1) ] """
            step2_feature_vector += numbering_token_using_2gram_lookup(j, j+1, sentence, word_bi_gram_lookup_table)

            FE_X_test[i][j] += step2_feature_vector # feature assignment

    # Wrtie feature information
    feature_info_without_LK.append('=> word bi-gram (t-1), (t) and dim = '+str(this_feature_length))
    feature_info_without_LK.append('=> word bi-gram (t), (t+1) and dim = '+str(this_feature_length))
    print('feature length 2-gram = ', this_feature_length)

feature length 2-gram =  1076


### Step 3. Using word tri-gram lookup table 

In [25]:
if using_3gram == 1:
    # for training data
    for i, sentence in enumerate(pre_X_train):
        for j, token in enumerate(sentence):

            step3_feature_vector = [] # [ 1g_t(t-1), 1g_t(t), 1g_t(t+1) ]

            """ [ token(t-2), token(t-1), token(t) ] """
            step3_feature_vector += numbering_token_using_3gram_lookup(j-2, j-1, j, sentence, word_tri_gram_lookup_table)
            """ [ token(t-1), token(t), token(t+1) ] """
            step3_feature_vector += numbering_token_using_3gram_lookup(j-1, j, j+1, sentence, word_tri_gram_lookup_table)
            """ [ token(t), token(t+1), token(t+2) ] """
            step3_feature_vector += numbering_token_using_3gram_lookup(j, j+1, j+2, sentence, word_tri_gram_lookup_table)

            FE_X_train[i][j] += step3_feature_vector # feature assignment  
            this_feature_length = len(step3_feature_vector)

    # for testing data
    for i, sentence in enumerate(pre_X_test):
        for j, token in enumerate(sentence):

            step3_feature_vector = [] # [ 1g_t(t-1), 1g_t(t), 1g_t(t+1) ]

            """ [ token(t-2), token(t-1), token(t) ] """
            step3_feature_vector += numbering_token_using_3gram_lookup(j-2, j-1, j, sentence, word_tri_gram_lookup_table)
            """ [ token(t-1), token(t), token(t+1) ] """
            step3_feature_vector += numbering_token_using_3gram_lookup(j-1, j, j+1, sentence, word_tri_gram_lookup_table)
            """ [ token(t), token(t+1), token(t+2) ] """
            step3_feature_vector += numbering_token_using_3gram_lookup(j, j+1, j+2, sentence, word_tri_gram_lookup_table)

            FE_X_test[i][j] += step3_feature_vector # feature assignment

    # Wrtie feature information
    feature_info_without_LK.append('=> word tri-gram (t-1), (t), (t+1) and dim = '+str(this_feature_length))
    print('feature length 3-gram = ', this_feature_length)

### Step 4. Using word five-gram lookup table 

In [26]:
if using_5gram == 1:
    # for training data
    for i, sentence in enumerate(pre_X_train):
        for j, token in enumerate(sentence):

            step4_feature_vector = [] # [ 1g_t(t-1), 1g_t(t), 1g_t(t+1) ]

            """ [ token(t-2), token(t-1), token(t), token(t+1), token(t+2) ] """
            step4_feature_vector += numbering_token_using_5gram_lookup(j-2, j-1, j, j+1, j+2, sentence, word_five_gram_lookup_table)

            FE_X_train[i][j] += step4_feature_vector # feature assignment 
            this_feature_length = len(step4_feature_vector)

    # for testing data
    for i, sentence in enumerate(pre_X_test):
        for j, token in enumerate(sentence):

            step4_feature_vector = [] # [ 1g_t(t-1), 1g_t(t), 1g_t(t+1) ]

            """ [ token(t-2), token(t-1), token(t), token(t+1), token(t+2) ] """
            step4_feature_vector += numbering_token_using_5gram_lookup(j-2, j-1, j, j+1, j+2, sentence, word_five_gram_lookup_table)

            FE_X_test[i][j] += step4_feature_vector # feature assignment

    # Wrtie feature information
    feature_info_without_LK.append('=> word five-gram (t-2), (t-1), (t), (t+1), (t+2) and dim = '+str(this_feature_length))
    print('feature length 5-gram = ', this_feature_length)

### Step 5. Using class lookup tables

In [27]:
if using_freqwords_eachclass == 1:
    # for training data
    for i, sentence in enumerate(pre_X_train):
        for j, token in enumerate(sentence):
            FE_X_train[i][j] += feature_extraction_1gram(j, sentence, lookup_table_class_component, window_size_class)
            FE_X_train[i][j] += feature_extraction_1gram(j, sentence, lookup_table_class_refinement_of_component, window_size_class)
            FE_X_train[i][j] += feature_extraction_1gram(j, sentence, lookup_table_class_action, window_size_class)
            FE_X_train[i][j] += feature_extraction_1gram(j, sentence, lookup_table_class_refinement_of_action, window_size_class)
            FE_X_train[i][j] += feature_extraction_1gram(j, sentence, lookup_table_class_condition, window_size_class)
            FE_X_train[i][j] += feature_extraction_1gram(j, sentence, lookup_table_class_priority, window_size_class)
            FE_X_train[i][j] += feature_extraction_1gram(j, sentence, lookup_table_class_motivation, window_size_class)
            FE_X_train[i][j] += feature_extraction_1gram(j, sentence, lookup_table_class_role, window_size_class)
            FE_X_train[i][j] += feature_extraction_1gram(j, sentence, lookup_table_class_object, window_size_class)
            FE_X_train[i][j] += feature_extraction_1gram(j, sentence, lookup_table_class_refinement_of_object, window_size_class)
            FE_X_train[i][j] += feature_extraction_1gram(j, sentence, lookup_table_class_sub_action, window_size_class)
            FE_X_train[i][j] += feature_extraction_1gram(j, sentence, lookup_table_class_sub_argument_of_action, window_size_class)
            FE_X_train[i][j] += feature_extraction_1gram(j, sentence, lookup_table_class_sub_priority, window_size_class)
            FE_X_train[i][j] += feature_extraction_1gram(j, sentence, lookup_table_class_sub_role, window_size_class)
            FE_X_train[i][j] += feature_extraction_1gram(j, sentence, lookup_table_class_sub_object, window_size_class)
            FE_X_train[i][j] += feature_extraction_1gram(j, sentence, lookup_table_class_sub_refinement_of_object, window_size_class)
            FE_X_train[i][j] += feature_extraction_1gram(j, sentence, lookup_table_class_none, window_size_class)
            
            
            
            
#             FE_X_train[i][j] += FE_for_classLUT(j, sentence) 
#             this_feature_length = len(FE_for_classLUT(j, sentence))

    # for testing data
    for i, sentence in enumerate(pre_X_test):
        for j, token in enumerate(sentence):
            FE_X_test[i][j] += feature_extraction_1gram(j, sentence, lookup_table_class_component, window_size_class)
            FE_X_test[i][j] += feature_extraction_1gram(j, sentence, lookup_table_class_refinement_of_component, window_size_class)
            FE_X_test[i][j] += feature_extraction_1gram(j, sentence, lookup_table_class_action, window_size_class)
            FE_X_test[i][j] += feature_extraction_1gram(j, sentence, lookup_table_class_refinement_of_action, window_size_class)
            FE_X_test[i][j] += feature_extraction_1gram(j, sentence, lookup_table_class_condition, window_size_class)
            FE_X_test[i][j] += feature_extraction_1gram(j, sentence, lookup_table_class_priority, window_size_class)
            FE_X_test[i][j] += feature_extraction_1gram(j, sentence, lookup_table_class_motivation, window_size_class)
            FE_X_test[i][j] += feature_extraction_1gram(j, sentence, lookup_table_class_role, window_size_class)
            FE_X_test[i][j] += feature_extraction_1gram(j, sentence, lookup_table_class_object, window_size_class)
            FE_X_test[i][j] += feature_extraction_1gram(j, sentence, lookup_table_class_refinement_of_object, window_size_class)
            FE_X_test[i][j] += feature_extraction_1gram(j, sentence, lookup_table_class_sub_action, window_size_class)
            FE_X_test[i][j] += feature_extraction_1gram(j, sentence, lookup_table_class_sub_argument_of_action, window_size_class)
            FE_X_test[i][j] += feature_extraction_1gram(j, sentence, lookup_table_class_sub_priority, window_size_class)
            FE_X_test[i][j] += feature_extraction_1gram(j, sentence, lookup_table_class_sub_role, window_size_class)
            FE_X_test[i][j] += feature_extraction_1gram(j, sentence, lookup_table_class_sub_object, window_size_class)
            FE_X_test[i][j] += feature_extraction_1gram(j, sentence, lookup_table_class_sub_refinement_of_object, window_size_class)
            FE_X_test[i][j] += feature_extraction_1gram(j, sentence, lookup_table_class_none, window_size_class)            
            

#             FE_X_test[i][j] += FE_for_classLUT(j, sentence) 
#     print('All class lookup table features = ', this_feature_length)

## Write out output files

In [28]:
# if last step,
feature_info_without_LK.append('****** Total feature dimension = '+str(len(FE_X_train[7][7])))
dump(FE_X_train, 'FE_X_train')
dump(FE_X_test, 'FE_X_test')
dump(feature_info_without_LK, 'feature_info_without_LK')
print('===> Total feature length =', len(FE_X_train[0][0]), len(FE_X_test[0][0]))

===> Total feature length = 9095 9095
